In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

from sklearn.metrics import accuracy_score

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ghckd\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
class PreProcessor:
    def __init__(self, folder_path:str="../data"):
        self.train = pd.read_csv(f"{folder_path}/train.csv")
        self.test = pd.read_csv(f"{folder_path}/test.csv")
        
    def data_preprocessor(self):
        train = self.train
        test = self.test

        train["text"] = train["text"].apply(self.text_cleaner)
        test["text"] = test["text"].apply(self.text_cleaner)
        
        # train = self.text_tr_processor(train)
        
        train["keyword"] = train["keyword"].fillna("")
        train["keyword_split"] = train["keyword"].apply(self.keyword_preprocessor)  
        train["keyword_split"] = train["keyword_split"].apply(self.lemmatizer)      

        test["keyword"] = test["keyword"].fillna("")
        test["keyword_split"] = test["keyword"].apply(self.keyword_preprocessor)  
        test["keyword_split"] = test["keyword_split"].apply(self.lemmatizer)      
                        
        # for i in train.index:
        #     train.loc[i, "text&keyword"] = train.loc[i, "text"] + " " + " ".join(train.loc[i, "keyword_split"])
        
        # for i in test.index:
        #     test.loc[i, "text&keyword"] = test.loc[i, "text"] + " " + " ".join(test.loc[i, "keyword_split"])
        
        # train = train.drop(columns=["id", "keyword", "location", "text", "keyword_split"])
        # test = test.drop(columns=["id", "keyword", "location", "text", "keyword_split"])
        
        train = train.drop(columns=["id", "keyword", "location", "keyword_split"]) 
        test = test.drop(columns=["id", "keyword", "location", "keyword_split"])

        
        return train, test
    
    def text_cleaner(self, text:str):
        if text is not np.nan:
            text = text.lower()
            text = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", text)
            stop = stopwords.words("english")
            text = " ".join([word for word in text.split() if word not in (stop)])
        
        return text

    def text_tr_processor(self, train_set):
        dup_texts = train_set[train_set.duplicated(["keyword", "text"], keep="first")]["text"].unique()
        drop_indexs = []
        for t in dup_texts:
            dup = train_set[train_set["text"] == t]
            if dup["target"].nunique() != 1:          
                rate_1 = train_set.loc[dup.index]["target"].sum()/len(train_set.loc[dup.index]["target"])
                if rate_1 < 0.5:
                    for i in dup.index:
                        train_set.loc[i, "target"] = 0
                elif rate_1 > 0.5:
                    for i in dup.index:
                        train_set.loc[i, "target"] = 1
                # elif rate_1 == 0.5:
                #     drop_indexs += list(dup.index)
        
        # train_set = train_set.drop(index=drop_indexs)
        train_set = train_set.drop_duplicates()
        return train_set

    def keyword_preprocessor(self, x:str):
        # if "hellfire" in x:
        #     x = "%20".join(("hell", "fire"))
        # if "wildfire" in x:
        #     x = "%20".join(("wild", "fire"))
        
        if "%20" in x:
            x = x.split("%20")
        else:
            x = [x]
    
        return x
    
    def lemmatizer(self, word_list):
        lemmatizer = nltk.wordnet.WordNetLemmatizer()
        for i, word in enumerate(word_list):
            word_list[i] = lemmatizer.lemmatize(word)
            word_list[i] = lemmatizer.lemmatize(word_list[i], "v")
            word_list[i] = lemmatizer.lemmatize(word_list[i], pos="a")
        return word_list

In [3]:
pp = PreProcessor()
train, test = pp.data_preprocessor()

In [4]:
lemmatizer = nltk.wordnet.WordNetLemmatizer()

def lemmatize_tokenizer(text):
    tokens = nltk.word_tokenize(text)    
    for i, token in enumerate(tokens):
        tokens[i] = lemmatizer.lemmatize(token)
        tokens[i] = lemmatizer.lemmatize(tokens[i], "v")
        tokens[i] = lemmatizer.lemmatize(tokens[i], pos="a")
    return tokens

In [5]:
vectorizer = CountVectorizer(tokenizer=lemmatize_tokenizer)
X_train = vectorizer.fit_transform(train["text"])
X_test = vectorizer.transform(test["text"])
y_train = train['target']

c:\Users\ghckd\anaconda3\envs\disaster-tweets\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [6]:
X_train.toarray().shape

(7613, 15265)

### modeling

#### RF + MNB + LR
* 0.9573098647051097
* 0.81244


In [7]:
rf_model = RandomForestClassifier(
    n_estimators=1000, 
    random_state=0, 
)
mnb_model = MultinomialNB()
lr_model = LogisticRegression(
    max_iter=100,
    random_state=0
)

voting_classifier = VotingClassifier(
    estimators=[('random_forest', rf_model), 
                ('multinomial_naive_bayes', mnb_model), 
                ('logistic_regression', lr_model), 
                ],
    voting='hard'
    )

voting_classifier.fit(X_train, y_train)
ensemble_train_pred = voting_classifier.predict(X_train)
ensemble_pred = voting_classifier.predict(X_test)

In [8]:
print("훈련 셋에 대한 정확도", accuracy_score(y_train, ensemble_train_pred))

훈련 셋에 대한 정확도 0.9573098647051097


In [9]:
ensemble_pred

array([1, 1, 1, ..., 1, 1, 0], dtype=int64)

In [11]:
submit = pd.read_csv("../submit/sample_submission.csv")
submit["target"] = ensemble_pred
# submit.to_csv("../submit/preprocessing_text_nodrop&keyword.csv", index=False)